**Testing time.sleep() for accurate camera acquisition during time lapses**

In [3]:
# functions used in the rest of the notebook

import time
import os
import random
import numpy as np

# normal use of time to add delay interval to timelapse
def time_test(expected_interval, duration, simulated_camera_delay):

    # Calculate the number of captures based on the duration and expected interval
    num_captures = int(duration / expected_interval)

    # Record the start time for reference
    start_time = time.time()

    captures = []
    for i in range(num_captures + 1):

        # simulated camera delay
        time.sleep(simulated_camera_delay)

        # Calculate the elapsed time from the start of the time-lapse
        elapsed_time = time.time() - start_time
        captures.append(elapsed_time)
        print(f"capture at: {elapsed_time:0.5f}")

        time.sleep(expected_interval)

    return(captures)

# adding adaptive time intervals after measuring camera acquisition
def adaptive_time_test(expected_interval, duration, simulated_camera_delay, add_noise=True):

    # Calculate the number of captures based on the duration and expected interval
    num_captures = int(duration / expected_interval)

    # Record the start time for reference
    start_time = time.time()

    captures = []
    for i in range(num_captures + 1):
        # Generate a unique filename for each capture
        #filename = f"{output_directory}/image_{i:04d}.jpg"

        # Capture an image and save it with the generated filename
        #camera.capture(filename)

        # simulated camera delay
        if(add_noise):
            simulated_camera_delay_rand = random.uniform(0.75*simulated_camera_delay, 1.5*simulated_camera_delay)
            time.sleep(simulated_camera_delay_rand)
        if(add_noise==False):
            time.sleep(simulated_camera_delay)

        # Calculate the elapsed time from the start of the time-lapse
        elapsed_time = time.time() - start_time
        captures.append(elapsed_time)
        print(f"capture at: {elapsed_time:0.5f}")

        # Calculate the expected time for the next capture
        expected_next_capture_time = (i + 1) * expected_interval

        # Calculate the time to sleep for the next capture
        sleep_duration = max(0, expected_next_capture_time - elapsed_time)

        # Wait for the adjusted delay before capturing the next image
        time.sleep(sleep_duration)

    return(captures)

In [8]:
expected_interval = 1  # Expected interval between captures (in seconds)
duration = 10  # Capture images for 20 seconds (20 images in this example)
delay = 0.2 # 200ms camera delay for testing

print('Capture timing with time.sleep() alone\n')
captures_control = time_test(expected_interval, duration, simulated_camera_delay=0)
print(f'\nMedian interval in control: {np.median(np.diff(captures_control)):0.5f} +/- {np.std(np.diff(captures_control)):0.5f}')

Capture timing with time.sleep() alone

capture at: 0.00001
capture at: 1.00507
capture at: 2.00564
capture at: 3.01075
capture at: 4.01631
capture at: 5.02163
capture at: 6.02701
capture at: 7.02739
capture at: 8.03294
capture at: 9.03848
capture at: 10.04320

Median interval in control: 1.00521 +/- 0.00194


Time.sleep() is actually quite accurate on the scale of seconds. We have experienced that it is inconsistent in practice though, suggesting there is a delay introduced by camera acquisition. Below I test what happens when we add a randomised camera delay between 150-300ms. I then used an adaptive interval between acquisitions to take into account the measured delay at each frame.

In [9]:
print('Capture timing with time.sleep() and simulated camera delay\n')
captures_delay = time_test(expected_interval, duration, simulated_camera_delay=delay)
print(f'\nMedian interval with camera delay: {np.median(np.diff(captures_delay)):0.5f} +/- {np.std(np.diff(captures_delay)):0.5f}')

Capture timing with time.sleep() and simulated camera delay

capture at: 0.20249
capture at: 1.41104
capture at: 2.62072
capture at: 3.83142
capture at: 5.04211
capture at: 6.24570
capture at: 7.45330
capture at: 8.66142
capture at: 9.87033
capture at: 11.07726
capture at: 12.28762

Median interval with camera delay: 1.20873 +/- 0.00205


You can see that the additive effect of camera delay over time results in a highly inaccurate timelapse. The last frame should be 10s, but is >12s

In [10]:
print('Capture timing with time.sleep() and simulated camera delay, but adaptive interval\n')
captures_delay_adaptive = adaptive_time_test(expected_interval, duration, simulated_camera_delay=delay)
print(f'Median adaptive interval with camera delay: {np.median(np.diff(captures_delay_adaptive)):0.5f} +/- {np.std(np.diff(captures_delay_adaptive)):0.5f}')

Capture timing with time.sleep() and simulated camera delay, but adaptive interval

capture at: 0.20413
capture at: 1.19445
capture at: 2.17737
capture at: 3.29813
capture at: 4.22819
capture at: 5.24864
capture at: 6.26934
capture at: 7.24535
capture at: 8.18754
capture at: 9.26127
capture at: 10.25171
Median adaptive interval with camera delay: 0.99038 +/- 0.05470


I used an adaptive interval here, which measures the camera delay and accounts for that in interval to next camera acquisition. This seems to prevent additive timing issues. The only downside is that the variability in interval to interval length is increased, but probably still at an acceptable level.